In [48]:
import pandas as pd

all_tweets_df = pd.read_csv('reduced_set')
all_tweets_df['label'] = all_tweets_df['label'].replace({2:1})
# all_tweets_df = all_tweets_df.dropna(axis=0)
# all_tweets_df['condition'] = all_tweets_df['condition'].replace({'depression':0, 'ptsd':1, 'control':2})
# all_tweets_df = all_tweets_df[pd.to_numeric(all_tweets_df['condition'], errors='coerce').notnull()]

In [61]:
all_tweets_df

,Unnamed: 0,text,label
0,0,This week is going to be perfect~,1
1,1,"@jkpAhe6LeZk Gracias for the follow, ginger!",1
2,2,@e9nx7G79OxSjgUX but you're mental age is abou...,0
3,3,@mrpqvocKBHYI stil gota b there for our teamm...,1
4,4,About to watch this honey boo boo child show.....,1
...,...,...,...
113221,113221,@e76GpcvHbF @e_BLroGB8Hl0w @mWEg_jcGU5z6pn4 @c...,0
113222,113222,@bKlxfsnv2iL6H did you go to the cincinnati one?,0
113223,113223,BLACK FUCKING METAL,0
113224,113224,"RT @xSs2zIUeUJ: DRAMA ON THE TL , DRAMA ON THE...",0


In [49]:
from sklearn.model_selection import train_test_split

# Training = 0.7, validation = 0.1, test = 0.2
training_tweets, temp_tweets, training_labels, temp_labels = train_test_split(list(all_tweets_df['text']),list(all_tweets_df['label']), test_size=0.3, random_state=123)
val_tweets, test_tweets, val_labels, test_labels = train_test_split(temp_tweets, temp_labels, test_size=(1/3), random_state=123)

In [50]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('prajjwal1/bert-tiny')

def tokenize_function(dataset):
    model_inputs = tokenizer(dataset['text'], padding="max_length", truncation=True, max_length=100)
    return model_inputs


loading file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/vocab.txt from cache at /Users/brookegrantham/.cache/huggingface/transformers/585ac1c3dedc6b808dd35e8770afafe10905d3e723a02617af749d39db780e09.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/config.json from cache at /Users/brookegrantham/.cache/huggingface/transformers/3cf34679007e9fe5d0acd644dcc1f4b26bec5cbc9612364f6da7262aed4ef7a4.a5a11219cf90aae61ff30e1658ccf2cb4aa84d6b6e947336556f887c9828dc6d
Model config BertConfig {
  "_name_or_path": "prajjwal1/bert-tiny",
  "attention_probs_dropout_prob": 

In [51]:
train = {'text': training_tweets, 'label': training_labels}
val = {'text': val_tweets, 'label': val_labels}
test = {'text': test_tweets, 'label': test_labels}

In [52]:
from datasets import Dataset

train_dataset = Dataset.from_dict(train)
val_dataset = Dataset.from_dict(val)
test_dataset = Dataset.from_dict(test)

Issue: There was NaN in the dataset not allowing it to be transfered to a Dataset type, thus checking the data condition in rough cells below.

In [53]:
# integers = [elm for elm in all_tweets_df['condition'] if not (isinstance(elm, int))]
# print(len(integers))

In [54]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

  0%|          | 0/80 [00:00<?, ?ba/s]

  0%|          | 0/23 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [55]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="transformer_checkpoints",
    num_train_epochs=1,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [56]:
from transformers import Trainer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('prajjwal1/bert-tiny', num_labels=2)

for param in model.bert.parameters():
    param.requires_grad = False


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

loading configuration file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/config.json from cache at /Users/brookegrantham/.cache/huggingface/transformers/3cf34679007e9fe5d0acd644dcc1f4b26bec5cbc9612364f6da7262aed4ef7a4.a5a11219cf90aae61ff30e1658ccf2cb4aa84d6b6e947336556f887c9828dc6d
Model config BertConfig {
  "_name_or_path": "prajjwal1/bert-tiny",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/pytorch_model.bin from cache at /Users/brookegrantham

Step,Training Loss


Saving model checkpoint to transformer_checkpoints/checkpoint-500
Configuration saved in transformer_checkpoints/checkpoint-500/config.json
Model weights saved in transformer_checkpoints/checkpoint-500/pytorch_model.bin
Saving model checkpoint to transformer_checkpoints/checkpoint-1000
Configuration saved in transformer_checkpoints/checkpoint-1000/config.json
Model weights saved in transformer_checkpoints/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to transformer_checkpoints/checkpoint-1500
Configuration saved in transformer_checkpoints/checkpoint-1500/config.json
Model weights saved in transformer_checkpoints/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to transformer_checkpoints/checkpoint-2000
Configuration saved in transformer_checkpoints/checkpoint-2000/config.json
Model weights saved in transformer_checkpoints/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to transformer_checkpoints/checkpoint-2500
Configuration saved in transformer_checkpoin

TrainOutput(global_step=9908, training_loss=0.6912006567599651, metrics={'train_runtime': 258.2099, 'train_samples_per_second': 306.952, 'train_steps_per_second': 38.372, 'total_flos': 19667238636000.0, 'train_loss': 0.6912006567599651, 'epoch': 1.0})

In [57]:
import numpy as np
import torch

def predict_nn(trained_model, test_dataset):

    output = trained_model(attention_mask=torch.tensor(test_dataset["attention_mask"]), input_ids=torch.tensor(test_dataset["input_ids"]))

    pred_labs = np.argmax(output["logits"].detach().numpy(), axis=1)

    gold_labs = test_dataset["label"]

    return gold_labs, pred_labs

gold_labs, pred_labs = predict_nn(model, test_dataset)

In [60]:
from sklearn.metrics import accuracy_score, f1_score
print("BERT-tiny on the John Hopkins Twitter dataset:")
print(f'The accuracy score is {accuracy_score(pred_labs, gold_labs)}')
print(f'The f1-score is {f1_score(pred_labs,gold_labs)}')


BERT-tiny on the John Hopkins Twitter dataset:
The accuracy score is 0.5428773293296829
The f1-score is 0.5482632222028277
